In [1]:
include("optimizer.jl")

optimize (generic function with 1 method)

In [3]:
b = 10
num_blocks = 200
n = num_blocks*(b-1) + 1

A = spzeros(n, n)

for i = 1:num_blocks
    top_left = (i-1)*(b-1) + 1
    A[top_left:top_left+b-1, top_left:top_left+b-1] .= randn(b, b)
end

A = (A + A')/2;